In [1]:

import torch

In [2]:
x = torch.tensor([1., 2.], requires_grad = True).view(-1,1)
print('x: ',x)

x:  tensor([[1.],
        [2.]], grad_fn=<ViewBackward0>)


**Class to define the neural network**

In [3]:
class Model(torch.nn.Module):
    def __init__(self) -> None:
        super(Model,self).__init__()
        self.layer01 = torch.nn.Linear(1,1)
    
    def forward(self,x):
        inputs = x
        output = torch.sigmoid(self.layer01(inputs))
        return output

In [4]:
model = Model()
print(model.state_dict())

OrderedDict([('layer01.weight', tensor([[-0.9171]])), ('layer01.bias', tensor([-0.2038]))])


In [5]:
u = model(x)

**First derivative**

In [6]:
u_x  = torch.autograd.grad(outputs=u, 
                           inputs=x,
                           create_graph=True,
                           grad_outputs=torch.ones_like(u)
                           )[0]
print(u_x)


tensor([[-0.1700],
        [-0.0935]], grad_fn=<MmBackward0>)


$$u(x; \theta)= \sigma(ax + b)$$
$$u_x(x; \theta)= \sigma'(ax + b)\frac{\partial}{\partial x}(ax + b) = \sigma(ax + b)(1 - \sigma(ax + b))a$$

In [7]:
# gradient check!
a = model.state_dict()['layer01.weight']
b = model.state_dict()['layer01.bias']
print(torch.sigmoid(a*x + b)*(1 - torch.sigmoid(a*x + b))*a)

tensor([[-0.1700],
        [-0.0935]], grad_fn=<MulBackward0>)


**Second derivative**

In [8]:
u_xx = torch.autograd.grad(outputs=u_x, 
                           inputs=x,
                           grad_outputs=torch.ones_like(u_x)
                           )[0]
print(u_xx)

tensor([[0.0793],
        [0.0660]])


$$u_x(x; \theta) = \sigma(ax + b)(1 - \sigma(ax + b))a$$
$$u_{xx}(x; \theta) = \sigma(ax + b)(1 - \sigma(ax + b))a^2 - \sigma(ax + b)^2(1 - \sigma(ax + b))a^2$$

In [9]:
# gradient check!
p1 = torch.sigmoid(a*x + b)*(1 - torch.sigmoid(a*x + b))**2*a**2
p2 = - torch.sigmoid(a*x + b)**2*(1 - torch.sigmoid(a*x + b))*a**2
print(p1 + p2)

tensor([[0.0793],
        [0.0660]], grad_fn=<AddBackward0>)


**Obs: If all the gradients are zero, then you get an error** (Don't know how to fix this yet)

In [10]:
class Linear_model(torch.nn.Module):
    def __init__(self) -> None:
        super(Linear_model,self).__init__()
        self.layer01 = torch.nn.Linear(1,1)
    
    def forward(self,x):
        inputs = x
        output = self.layer01(inputs)
        return output
#
linear_model = Linear_model()
print(linear_model.state_dict())
#
lm = linear_model(x)
#
lm_x  = torch.autograd.grad(outputs=lm, 
                            inputs=x,
                            create_graph=True,
                            grad_outputs=torch.ones_like(lm)
                            )[0]
print(lm_x)
#
lm_xx = torch.autograd.grad(outputs=lm_x, 
                            inputs=x,
                            grad_outputs=torch.ones_like(lm_x)
                            )[0]
print(lm_xx)


OrderedDict([('layer01.weight', tensor([[-0.4659]])), ('layer01.bias', tensor([-0.1514]))])
tensor([[-0.4659],
        [-0.4659]], grad_fn=<MmBackward0>)


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.